In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_chamado_log"

nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

In [0]:
query = f"""
CREATE OR REPLACE TABLE {nome_tabela_destino} (
    cd_chamado BIGINT COMMENT 'Referência ao chamado principal.',
    cd_cliente STRING COMMENT 'Referência ao cliente.',
    dh_abertura TIMESTAMP COMMENT 'Timestamp de abertura.',
    dh_inicio TIMESTAMP COMMENT 'Timestamp de início.',
    dh_fim TIMESTAMP COMMENT 'Timestamp de fim.',
    dt_ingestion TIMESTAMP,
    dc_origem STRING,
    CONSTRAINT pk_chamado_log PRIMARY KEY (cd_chamado)
)
COMMENT 'Tabela silver de logs de horas (abertura, início e fim) de chamados';
"""

spark.sql(query)
print(f"✅ Tabela {nome_tabela_destino} criada com sucesso!")